In [ ]:
# parâmetros iniciais
a = 2  # Aceleração
v = 280 #velocidade
t = 1  # tempo

In [ ]:
import numpy as np
from numpy.linalg import inv

x_medicoes = np.array([4000, 4550, 4860, 5110])
v_medicoes = np.array([280, 285, 286, 290])

z = np.c_[x_medicoes, v_medicoes]

In [ ]:
# erros
error_est_x = 30
error_est_v = 10

# erros medição
error_obs_x = 25  # incerteza de medição
error_obs_v = 6

In [ ]:
def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


In [ ]:
# Matriz de covariância estimação inicial
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Matriz de estado inicial
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

In [ ]:
for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Ganho de Kalman
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # reshape dos dados
    Y = H.dot(data).reshape(n, -1)

    # Atualização da matriz de estado
    # Combinação entre o estado predito, valores medidos, matriz de covariância e ganho de Kalman
    G = X + K.dot(Y - H.dot(X))

    # Atualizar matriz de covariância do processo
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

print("Modelo previsto:\n", X)
print("Matriz de Estado:\n", G)

Modelo previsto:
 [[4849.]
 [ 286.]]
Matriz de Estado:
 [[4926.34914588]
 [ 287.19047619]]


In [ ]:
Y

array([[5110.],
       [ 290.]])